 🤖 My First Groq-Powered Coding Assistant (Notebook Edition)

Welcome! This Jupyter Notebook will guide you through creating a simple coding assistant that uses the Groq API for fast responses from Large Language Models (LLMs) like Llama 3.

**What you'll learn:**
- How to install and import necessary Python libraries.
- How to securely configure your Groq API key.
- How to connect to the Groq service.
- How to send prompts to an LLM and get responses.
- How to use "tools" (like getting the current time) with the LLM.
- How to evaluate the LLM's responses.
- How to create a basic interactive chat loop.

Let's get started!

## ⚙️ Step 0: Setup - Install Libraries

First, we need to make sure we have the `groq` library (for interacting with the Groq API) and `python-dotenv` (for managing our API key securely).

**Instructions:**
1. Run the code cell below **once**.
2. If the libraries are already installed, it won't do any harm.
3. After running it successfully, you can comment out the `!pip install` lines (by adding a `#` at the beginning of those lines) to avoid running them every time you open the notebook.

In [ ]:
# Before running this, make sure you have Python installed.
# You can run these commands in your terminal or directly in a notebook cell by adding "!" at the beginning.
# If you run them here, they only need to be run once.

# print("Installing necessary libraries...")
# !pip install groq python-dotenv

# print("Installation complete! You can now comment out the !pip install lines or remove this cell.")

### Understanding F Strings and Format Functions

In [18]:
# Let us understand what is f string and .format() function as well. 

# Simple Example
name = "Alice"
greeting = f"Hello, {name}!"
print(greeting)

greeting = 'Hello, {name}'
print(greeting.format(name='Bhavishya'))

message = 'I am learning {} and {} today!'
print(message.format('f-string', 'format function'))

# f string with function call
def convert_to_upper(text):
    return text.upper()

name = "data science"
print(f"Course name: {convert_to_upper(name)}")


# f strings with dictionary
data = {"name": "Dana", "task": "Analysis", "hours": 4}
template = "Employee {name} worked on {task} for {hours} hours."
print(template.format(**data))

Hello, Alice!
Hello, Bhavishya
I am learning f-string and format function today!
Course name: DATA SCIENCE
Employee Dana worked on Analysis for 4 hours.


### Hands-on Activity - Format Function and F-String

In [ ]:
# Ask the user for a number using input(). Then use an f-string and format function to print the number squared.

In [ ]:
# Write a function greet(name, role) that returns a greeting using f-strings and format():
# greet("Tina", "Manager") ➞ "Hello Tina! You are working as a Manager."

Now, let's import the libraries we'll need for this project.

In [1]:
import os
import json
from groq import Groq, RateLimitError, APIError
from datetime import datetime
from dotenv import load_dotenv # For loading API key from a .env file

print("Libraries imported successfully!")

Libraries imported successfully!


## 🔑 Step 1: Configure Your Groq API Key

To use the Groq API, you need an API key. You can get one from the [Groq Console](https://console.groq.com/keys).


**For Local Jupyter:**
1.  **Create a file named `.env`** in the *same directory* as this notebook.
2.  Open the `.env` file with a text editor.
3.  Add the following line, replacing `your_gsk_key_here` with your actual Groq API key:
    ```
    GROQ_API_KEY="your_gsk_key_here"
    ```
4.  Save the `.env` file.

**For Google Colab (Preferred for Colab):**
1. Click the **key icon (🔑)** in the left sidebar of Colab.
2. Click `+ ADD A NEW SECRET`.
3. Name: `GROQ_API_KEY`
4. Value: Your actual Groq API key.
5. Ensure 'Notebook access' is ON.
The code in the next cell is set up for local `.env` but will need adjustment for Colab secrets (see Colab instructions I provided earlier).

In [2]:
# Load environment variables from .env file (for local use)
# For Colab, you'd use: from google.colab import userdata; GROQ_API_KEY = userdata.get('GROQ_API_KEY')
load_dotenv()

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

if not GROQ_API_KEY:
    print("GROQ_API_KEY not found. Ensure it's in .env (local) or Colab Secrets and you've adapted this cell if in Colab.")
    # Fallback for manual input if needed, but not recommended for routine use:
    # GROQ_API_KEY = input("Please enter your Groq API Key: ")
    if not GROQ_API_KEY: # Check again if it was entered
        raise ValueError(
            "Groq API Key is not set. Please configure it and restart the kernel."
        )
else:
    print("Groq API Key loaded (or attempted from environment/Colab Secrets).")

# --- Configuration ---
DEFAULT_MODEL = "llama3-8b-8192"
EVALUATION_MODEL = "llama3-8b-8192" # Can be the same or different

print(f"Using model: {DEFAULT_MODEL}")
print(f"Using evaluation model: {EVALUATION_MODEL}")

Groq API Key loaded (or attempted from environment/Colab Secrets).
Using model: llama3-8b-8192
Using evaluation model: llama3-8b-8192


## 🔗 Step 2: Connect to the Groq Client

Now that we have the API key, let's write a function to initialize the Groq client. This client is what we'll use to communicate with the Groq API.

In [20]:
def get_groq_client():
    """Initializes and returns the Groq client."""
    
    try:
        client = Groq(api_key=GROQ_API_KEY)
        print("Groq client initialized successfully.")

        # --- LLM call demo ---
        try:
            response = client.chat.completions.create(
                model=DEFAULT_MODEL,
                messages=[{"role": "user", "content": "Say hello! This is a test LLM call from get_groq_client."}],
                temperature=0.5,
                max_tokens=200
            )
            print("LLM Test Call Response:", response.choices[0].message.content)
        except Exception as llm_error:
            print("LLM call failed:", llm_error)
        # --- End demo ---

        return client
    except APIError as e:
        print(f"Failed to initialize Groq client due to APIError: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during client initialization: {e}")
        return None

# Attempt to initialize the client
groq_client = get_groq_client()

if groq_client:
    print("Successfully connected to Groq!")
else:
    print("Failed to connect to Groq. Please check your API key and network connection.")

Groq client initialized successfully.
LLM Test Call Response: Hello! It's nice to meet you! I'm here to help with any questions or tasks you may have. What can I assist you with today?
Successfully connected to Groq!


## 🛠️ Step 3: Define Helper Functions

We need a few helper functions:
1.  `get_current_datetime`: An example function that the LLM can "call" if it needs the current time to answer a coding question.
2.  `filter_messages_for_api`: This function ensures we only send data to the API that it expects, removing any custom keys we might add for our own use (like evaluation results).

In [21]:
def get_current_datetime():
    """Returns the current date and time as a JSON string."""
    return json.dumps({"current_datetime": datetime.now().isoformat()})

def filter_messages_for_api(messages):
    api_messages = []
    for msg in messages:
        api_msg = msg.copy()
        api_msg.pop("evaluation", None)
        api_messages.append(api_msg)
    return api_messages

print("Helper functions defined.")

# --- Demo: Use the helper functions with a real LLM call ---
groq_client = get_groq_client()  # Reuse the enhanced client setup

if groq_client:
    # Get current datetime
    datetime_json = get_current_datetime()
    print("Current datetime JSON:", datetime_json)

    # Construct message with flavor
    raw_messages = [
        {
            "role": "user",
            "content": f"Can you greet the user and also acknowledge the current time: {datetime_json}?",
            "evaluation": "demo"  # This key should be stripped
        }
    ]
    
    filtered_messages = filter_messages_for_api(raw_messages)
    print("Filtered messages for LLM API:", filtered_messages)

    try:
        response = groq_client.chat.completions.create(
            model=DEFAULT_MODEL,
            messages=filtered_messages
        )
        print("LLM Response:", response.choices[0].message.content)
    except Exception as e:
        print("LLM call failed:", e)
else:
    print("Skipping LLM call demo due to client initialization failure.")


Helper functions defined.
Groq client initialized successfully.
LLM Test Call Response: Hello! It's great to hear from you! I'm here to assist with any questions or tasks you may have.
Current datetime JSON: {"current_datetime": "2025-05-10T21:17:10.329672"}
Filtered messages for LLM API: [{'role': 'user', 'content': 'Can you greet the user and also acknowledge the current time: {"current_datetime": "2025-05-10T21:17:10.329672"}?'}]
LLM Response: What a lovely request!

Here's a warm greeting and a nod to the current time:

**Hello! It's great to chat with you! **

As I check the current time, I see that it's currently **May 10th, 2025 at 9:17 PM**.


In [22]:
# Tool function 1
def get_current_datetime():
    """Returns the current date and time as a JSON string."""
    return json.dumps({"current_datetime": datetime.now().isoformat()})

# Tool function 2 (a dummy example)
def get_welcome_message(name="User"):
    """Returns a friendly welcome message."""
    return f"Hello, {name}! Welcome to your AI assistant demo."

# Simulated tool registry
tool_functions = {
    "get_current_datetime": get_current_datetime,
    "get_welcome_message": lambda: get_welcome_message("Bhavishya")
}

# Simulated question that will trigger function call logic
user_question = "Can you tell me the current time?"

# Get Groq client
groq_client = get_groq_client()

if groq_client:
    # Send the user question to Groq to determine which function to call
    routing_response = groq_client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that decides which function to call. Only return the name of the function nothing else."},
            {"role": "user", "content": f"Question: {user_question}\n"
                                        f"Available functions: get_current_datetime, get_welcome_message\n"
                                        f"Reply ONLY with the function name to call (no extra words)."}
        ]
    )

    chosen_function = routing_response.choices[0].message.content.strip()
    print(f"LLM chose to call: {chosen_function}")

    # Call the corresponding function if valid
    if chosen_function in tool_functions:
        result = tool_functions[chosen_function]()
        print(f"Function '{chosen_function}' result:", result)

        # Send the result back to the LLM for response generation
        final_response = groq_client.chat.completions.create(
            model=DEFAULT_MODEL,
            messages=[
                {"role": "user", "content": f"The result of {chosen_function} is: {result}. "
                                            f"Can you explain this nicely to the user?"}
            ]
        )
        print("LLM Final Explanation:", final_response.choices[0].message.content)
    else:
        print("Invalid function name returned by LLM.")
else:
    print("Groq client initialization failed. Cannot demonstrate function calling.")


Groq client initialized successfully.
LLM Test Call Response: Hello! It's great to hear from you! I'm here to help with any questions or topics you'd like to discuss. It looks like this is a test call from get_groq_client, is that correct? If you're looking to test my language understanding or just want to chat, I'm happy to oblige. What's on your mind?
LLM chose to call: get_current_datetime
Function 'get_current_datetime' result: {"current_datetime": "2025-05-10T21:17:27.122378"}
LLM Final Explanation: The result of `get_current_datetime` is a Python dictionary that contains a single key-value pair. The key is `"current_datetime"`, and the value is a string that represents the current date and time.

In this particular case, the value is the string `"2025-05-10T21:17:27.122378"`.

Let's break it down:

* `2025-05-10`: This is the date in the format "Year-Month-Day".
* `T`: This is a literal character that separates the date from the time.
* `21:17:27`: This is the time in the format 

## 💬 Step 4: Adding Memory

In [23]:
# Memory: store all conversation turns here
chat_history = []

# Simulated incoming question from user
user_question = "Can you tell me the current time?"

# Get Groq client
groq_client = get_groq_client()

if groq_client:
    # Append user question to history
    chat_history.append({"role": "user", "content": user_question})

    # System prompt to instruct the LLM
    system_prompt = {
        "role": "system",
        "content": (
            "You are a helpful assistant that decides which function to call. "
            "Reply ONLY with the function name (e.g., get_current_datetime), nothing else."
        )
    }

    # Add the routing query to chat
    routing_messages = chat_history.copy()
    routing_messages.insert(0, system_prompt)
    routing_messages.append({
        "role": "user",
        "content": f"Available functions: {', '.join(tool_functions.keys())}\n"
                   f"Decide which one to call based on the last user input."
    })

    # Ask LLM to route the function
    routing_response = groq_client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=routing_messages
    )

    chosen_function = routing_response.choices[0].message.content.strip()
    print(f"LLM chose to call: {chosen_function}")

    # Save assistant response (function name)
    chat_history.append({"role": "assistant", "content": chosen_function})

    if chosen_function in tool_functions:
        result = tool_functions[chosen_function]()
        print(f"Function '{chosen_function}' result:", result)

        # Add the function result as a message to the history
        chat_history.append({
            "role": "function",
            "name": chosen_function,
            "content": result
        })

        # Now let LLM summarize/explain this in a friendly way
        final_response = groq_client.chat.completions.create(
            model=DEFAULT_MODEL,
            messages=chat_history + [
                {
                    "role": "user",
                    "content": f"The result of {chosen_function} was returned. Please explain that to the user nicely."
                }
            ]
        )

        friendly_output = final_response.choices[0].message.content
        print("LLM Final Explanation:", friendly_output)

        # Save final assistant message
        chat_history.append({"role": "assistant", "content": friendly_output})
    else:
        print("Invalid function name returned by LLM.")
else:
    print("Groq client initialization failed. Cannot demonstrate function calling.")

Groq client initialized successfully.
LLM Test Call Response: Hello! It's great to hear from you! I'm here to help with any questions or tasks you may have.
LLM chose to call: get_current_datetime
Function 'get_current_datetime' result: {"current_datetime": "2025-05-10T21:17:41.665088"}
LLM Final Explanation: I see what you did there!

I'm a large language model, I don't have direct access to the current time, but I can try to help you with that.

When you called the `get_current_datetime` function, it returned the current date and time in a JSON format, which looks like this: `{"current_datetime": "2025-05-10T21:17:41.665088"}`.

This format is a standard way of representing dates and times in computers. Let me break it down for you:

* The `"current_datetime"` part is the label for the current date and time.
* The value that follows, `"2025-05-10T21:17:41.665088"`, is the actual date and time.

Here's what each part of the value represents:
	+ `"2025-05-10"` is the date in the format

### Stateful AI

In [28]:
from datetime import datetime

# --- Tool Functions ---
def calculate_sum(a, b):
    return f"The sum of {a} and {b} is {a + b}."

def get_user_profile(name="Guest"):
    return f"{name} is a curious learner exploring AI tools!"

def give_motivational_quote():
    return "Believe in yourself. Every expert was once a beginner."

# --- Tool Registry ---
tool_functions = {
    "calculate_sum": lambda: calculate_sum(7, 5),
    "get_user_profile": lambda: get_user_profile("Bhavishya"),
    "give_motivational_quote": give_motivational_quote
}

# --- Memory ---
chat_history = []
tool_usage_history = []

# --- Groq Client ---
groq_client = get_groq_client()

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "end"]:
        break

    chat_history.append({"role": "user", "content": user_input})

    # --- Tool Routing Phase ---
    routing_prompt = [
        {"role": "system", "content": (
            "You are a smart assistant with tools: calculate_sum, get_user_profile, give_motivational_quote.\n"
            "If the user input requires one of these tools, respond ONLY with the tool name.\n"
            "If it doesn't, respond with 'none'."
        )}
    ] + chat_history[-3:] + [
        {"role": "user", "content": "Which tool should be used now? Or reply 'none'."}
    ]

    routing_response = groq_client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=routing_prompt
    )

    function_decision = routing_response.choices[0].message.content.strip()
    chat_history.append({"role": "assistant", "content": function_decision})

    if function_decision in tool_functions:
        result = tool_functions[function_decision]()
        print(f"[Tool: {function_decision}] {result}")

        tool_usage_history.append({
            "tool": function_decision,
            "timestamp": datetime.now().isoformat(),
            "result": result
        })

        chat_history.append({
            "role": "function",
            "name": function_decision,
            "content": result
        })

    else:
        # --- General LLM Response with Memory Awareness ---

        # 💡 Fix: move memory to TOP system message
        memory_context = "\n".join([
            f"- {entry['tool']} → {entry['result']} at {entry['timestamp']}"
            for entry in tool_usage_history
        ]) or "No tools used yet."

        full_prompt = [
            {
                "role": "system",
                "content": (
                    f"You are a helpful assistant. Here is the user's tool usage history:\n"
                    f"{memory_context}\n"
                    f"Answer based on current message and history."
                )
            }
        ] + chat_history

        general_response = groq_client.chat.completions.create(
            model=DEFAULT_MODEL,
            messages=full_prompt
        )

        output = general_response.choices[0].message.content
        print("Assistant:", output)
        chat_history.append({"role": "assistant", "content": output})

Groq client initialized successfully.
LLM Test Call Response: Hello! It's great to hear from you. I'm here to help with any questions or tasks you may have. Please feel free to ask me anything.
[Tool: calculate_sum] The sum of 7 and 5 is 12.
Assistant: , you called `calculate_sum` with arguments 7 and 5 once


## 📊 Step 5: Evaluating the Assistant's Response

This function will use another LLM call to assess the relevance and helpfulness of the assistant's response.

In [ ]:
def evaluate_response(client, user_query, assistant_response_content, model):
    if not client:
        print("Groq client is not initialized. Cannot evaluate response.")
        return "Evaluation failed (client not initialized)."

    eval_system_prompt_content = f"""You are an evaluation AI. Evaluate the assistant's response based on the user's query.
    User Query: "{user_query}"
    Assistant Response: "{assistant_response_content}"

    Evaluate based on these criteria:
    1.  **Coding Relevance:** Was the assistant's response strictly related to coding/programming topics? (Yes/No)
    2.  **Helpfulness (if relevant):** If the response was coding-related, how helpful and accurate was it? (Score 1-5, 5=Excellent, 1=Not Helpful, NA if not relevant)
    3.  **Refusal Appropriateness (if irrelevant):** If the user's query was *not* coding-related, did the assistant politely refuse according to its instructions? (Yes/No/NA)

    Provide the evaluation concisely, starting with "Evaluation:".
    Example (Relevant): "Evaluation: Coding Relevance: Yes, Helpfulness: 4/5, Refusal Appropriateness: NA"
    Example (Irrelevant, Correct Refusal): "Evaluation: Coding Relevance: No, Helpfulness: NA, Refusal Appropriateness: Yes"
    """
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": eval_system_prompt_content}],
        temperature=0.1,
    )
    return response.choices[0].message.content

print("`evaluate_response` function defined.")
sample_user_query = "How do I reverse a string in Python?"

sample_response = groq_client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=[
            {"role": "user", "content": sample_user_query}
        ]
    )
print('LLM Response:', sample_response)
print(evaluate_response(groq_client, sample_user_query, sample_response, DEFAULT_MODEL))

`evaluate_response` function defined.
Evaluation: Coding Relevance: Yes, Helpfulness: 5/5, Refusal Appropriateness: NA


## 🎉 Congratulations!

You've successfully set up and interacted with a Groq-powered Coding Assistant!

**Next Steps & Ideas:**
- Try different coding questions.
- Ask a non-coding question to see the refusal.
- Experiment with models and tools.